In [2]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit_algorithms.optimizers import COBYLA
import itertools
from collections import Counter

# FOOSP 参数 (One-hot)
n = 2  # 作业数
m = 3  # 机器数
l = 3  # C_max 比特数
p = np.array([[1, 2, 3], [2, 3, 1]])  # 处理时间
W = [2]  # 不可用机器集合
K1 = 100  # 不可用惩罚系数
P = 200  # 负载惩罚系数
Q = 1000  # 辅助约束惩罚系数
num_qubits_onehot = n * m + l + m  # 总比特数


# 定义量子门
def append__z_term(qc, qubit, gamma):
    qc.rz(2 * gamma, qubit)


def append__zz_term(qc, qubit1, qubit2, gamma):
    qc.cx(qubit1, qubit2)
    qc.rz(2 * gamma, qubit2)
    qc.cx(qubit1, qubit2)


def append__mixer_term(qc, qubit, beta):
    qc.rx(-2 * beta, qubit)


# H_1：C_max
def get_cost_circuit1(gamma, qc):
    for h in range(l):
        coef = -0.5 * 2 ** h  # 调整为量子形式
        append__z_term(qc, n * m + h, gamma * coef)
    return qc


# H_2：不可用机器惩罚
def get_cost_circuit2(gamma, qc):
    for j in W:
        for i in range(n):
            append__z_term(qc, i * m + j, K1 * gamma)  # x_{ij} = (1 - Z_{ij})/2
    return qc


# H_3：负载惩罚 with a_j
def get_cost_circuit3(gamma, qc):
    aux_start = n * m + l
    for j in range(m):
        # a_j (C_j - C_max)^2
        for i in range(n):
            for i_prime in range(n):
                coef = P * p[i, j] * p[i_prime, j]
                append__zz_term(qc, i * m + j, i_prime * m + j, gamma * coef)  # C_j^2
                append__zz_term(qc, i * m + j, aux_start + j, gamma * coef)  # a_j C_j^2
            constant = P * p[i, j]
            for h in range(l):
                coef = constant * 2 ** h
                append__zz_term(qc, i * m + j, n * m + h, -2 * gamma * coef)  # -2 C_j C_max
                append__zz_term(qc, i * m + j, aux_start + j, gamma * coef)  # a_j terms
                append__zz_term(qc, n * m + h, aux_start + j, -2 * gamma * coef)
        for h in range(l):
            for h_prime in range(l):
                coef = 0.25 * P * 2 ** h * 2 ** h_prime
                append__zz_term(qc, n * m + h, n * m + h_prime, gamma * coef)  # C_max^2
                append__zz_term(qc, n * m + h, aux_start + j, gamma * coef)  # a_j C_max^2

        # a_j (C_max - C_j)^2
        for i in range(n):
            for i_prime in range(n):
                coef = Q * p[i, j] * p[i_prime, j]
                append__zz_term(qc, i * m + j, i_prime * m + j, gamma * coef)
                append__zz_term(qc, i * m + j, aux_start + j, gamma * coef)
            constant = Q * p[i, j]
            for h in range(l):
                coef = constant * 2 ** h
                append__zz_term(qc, i * m + j, n * m + h, -2 * gamma * coef)
                append__zz_term(qc, i * m + j, aux_start + j, gamma * coef)
                append__zz_term(qc, n * m + h, aux_start + j, -2 * gamma * coef)
    return qc


# 混合哈密顿量
def get_mixer_circuit(beta, qc):
    for i in range(qc.num_qubits):
        append__mixer_term(qc, i, beta)
    return qc


# 反转比特序
def invert_counts(s):
    return s[::-1]


# 成本函数
def cost_function_onehot(params):
    beta, gamma = params[:len(params) // 2], params[len(params) // 2:]
    qc = QuantumCircuit(num_qubits_onehot, num_qubits_onehot)
    qc.h(range(num_qubits_onehot))
    for i in range(len(beta)):
        get_cost_circuit1(gamma[i], qc)
        get_cost_circuit2(gamma[i], qc)
        get_cost_circuit3(gamma[i], qc)
        get_mixer_circuit(beta[i], qc)
    qc.measure(range(num_qubits_onehot), range(num_qubits_onehot))
    backend = Aer.get_backend('qasm_simulator')
    shots = 10000
    result = execute(qc, backend, seed_simulator=10, shots=shots).result()
    counts = result.get_counts(qc)

    expectation = 0
    for bitstring, count in counts.items():
        bitstring = invert_counts(bitstring)
        prob = count / shots
        x = np.array([int(b) for b in bitstring[:n * m]]).reshape(n, m)
        z_h = np.array([int(b) for b in bitstring[n * m:n * m + l]])
        a_j = np.array([int(b) for b in bitstring[n * m + l:]])

        E1 = np.sum([2 ** h * z_h[h] for h in range(l)])
        E2 = K1 * sum(x[i, j] for j in W for i in range(n))
        E3 = 0
        for j in range(m):
            C_j = sum(p[i, j] * x[i, j] for i in range(n))
            C_max = E1
            delta = C_j - C_max
            E3 += P * a_j[j] * delta ** 2 + Q * a_j[j] * (C_max - C_j) ** 2

        E = E1 + E2 + E3
        expectation += E * prob

    return expectation, counts

# 参数优化和主程序（与 Binary 相同，略）

In [4]:
def optimize_qaoa(params, cost_func):
    optimizer = COBYLA(rhobeg=1.5, tol=1e-7)
    return optimizer.minimize(fun=lambda p: cost_func(p)[0], x0=params)


def fourier_initial_params(p, q=1, amplitude=0.5):
    t = np.linspace(0, 1, p)
    beta = np.zeros(p)
    gamma = np.zeros(p)
    for k in range(1, q + 1):
        beta += np.random.uniform(0, amplitude) * np.cos(2 * np.pi * k * t)
        gamma += np.random.uniform(0, amplitude) * np.sin(2 * np.pi * k * t)
    beta = np.clip(beta, 0, np.pi)
    gamma = np.clip(gamma, 0, 2 * np.pi)
    return np.concatenate([beta, gamma])


import time

for encoding, cost_func, num_qubits in [("One-hot", cost_function_onehot, num_qubits_onehot)]:
    start_time = time.time()
    min_energy = float('inf')
    for p1 in range(1, 4):
        for k in range(5):
            print(f"{encoding} - p={p1}, iter={k}")
            init_point = fourier_initial_params(p1, q=1, amplitude=0.5)
            result = optimize_qaoa(init_point, cost_func)
            energy, counts = cost_func(result.x)
            if energy < min_energy:
                min_energy = energy
                opt_bitstring = min(counts.keys(), key=lambda b: cost_func([result.x])[0])
                opt_prob = counts.get(opt_bitstring, 0) / 10000
                print(f"Energy: {energy}, P_opt: {opt_prob}")
    print(f"{encoding} - Total time: {time.time() - start_time} s, Min Energy: {min_energy}")

One-hot - p=1, iter=0


CircuitError: 'duplicate qubit arguments'